In [ ]:
print("Hello world")

Hello world


In [7]:
!pip install --quiet transformers
!pip install --quiet tensorboard
!pip install --quiet  sentence_transformers
import pandas as pd
import numpy as np
import torch
import pandas as pd
from time import time
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import tensorflow_hub as hub # For USE
from sentence_transformers import SentenceTransformer
import os
import tensorflow as tf
import datetime
from tensorboard.plugins import projector


def generate_embeddings(model_name, data_ls):
    if model_name == "USE":
        print("Check 1")
        embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
        print("Check 2")
        embeddings = embed(data_ls)
        print("Check 3")
        embeddings = embeddings.numpy()
        print("Check 4")
        return embeddings

    elif model_name == "all-MiniLM":
        # All-round model tuned for many use-cases. Trained on a large and diverse dataset of over 1 billion training pairs.
        model = SentenceTransformer('all-MiniLM-L12-v2')
        embeddings = model.encode(data_ls)
        return embeddings

    elif model_name == "paraphrase-MiniLM-L6-v2":
        model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
        embeddings = model.encode(data_ls)
        return embeddings

    elif model_name == "multi-qa-MiniLM-L6-cos-v1":
        # This model was tuned for semantic search: Given a query/question, if can find relevant passages. It was trained on a large and diverse set of (question, answer) pairs.
        model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
        embeddings = model.encode(data_ls)
        return embeddings

    elif model_name == "multi-qa-mpnet-base-dot-v1":
        # This model was tuned for semantic search: Given a query/question, if can find relevant passages. It was trained on a large and diverse set of (question, answer) pairs.
        model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
        embeddings = model.encode(data_ls)
        return embeddings
    elif model_name == "all-mpnet-base-v2":
        # All-round model tuned for many use-cases. Trained on a large and diverse dataset of over 1 billion training pairs.
        model = SentenceTransformer('all-mpnet-base-v2')
        embeddings = model.encode(data_ls)
        return embeddings

    elif model_name == "paraphrase-albert-small-V2":
        model = SentenceTransformer('paraphrase-albert-small-v2')
        embeddings = model.encode(data_ls)
        return embeddings

    elif model_name == "all-distilroberta-v1":
        model = SentenceTransformer('all-distilroberta-v1')
        embeddings = model.encode(data_ls)
        return embeddings

    else:
        print("sorry")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/snapshot_download.py:11: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  FutureWarning,


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [9]:
df = pd.read_csv("drive/MyDrive/Extramarks Project/Dataset versions/JEE-with-answers-without-numericals.csv").drop("Unnamed: 0", axis = "columns")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
len1 = len(df)
df = df.dropna()
print("Dropped ", str(len(df) - len1) , " rows")

Dropped  -581  rows


In [11]:
len(df)

133850

In [12]:
rows_to_leave_ls = []
t = time()
index= 0
for ans in df['answer'].str.split():
  if index%20000 == 0:
    print(index)  
  if "stryde" in ans and "images" in ans:
    rows_to_leave_ls.append(False)
  elif "ckfinder" in ans and "userfiles" in ans :
    rows_to_leave_ls.append(False)
  else:
    rows_to_leave_ls.append(True)
  index+=1

print("Time taken: " , int((time() - t)%60) ,"s, ", int((time() - t)/60), "m")

0
20000
40000
60000
80000
100000
120000
Time taken:  0 s,  0 m


In [13]:
df = df[rows_to_leave_ls]
len(df)

131123

In [14]:
# df.to_csv("drive/MyDrive/Extramarks Project/Dataset versions/JEE-with-answers-without-numericals-removed_Image_Answers.csv")

In [15]:
# df = df[:1000]

In [16]:
count = 0
question_sentences = []
answer_sentences= []


for index in range(len(df)):
  i = df.iloc[index]['question']
  ans = df.iloc[index]['answer']

  if  type(i) != type("") or len(i) <= 0 or type(ans) != type("") or len(ans) <= 0:
    count+=1
  else:
    question_sentences.append(i)
    answer_sentences.append(ans)

print(count, " sentences removed.")

0  sentences removed.


##### Generating embeddings for questions

Next cell takes about 5 m to run

In [ ]:
t =time()
embeds = generate_embeddings("all-mpnet-base-v2", question_sentences)
print("Time taken: " , int((time() - t)%60) ,"s, ", int((time() - t)/60), "m")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/file_download.py:563: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Time taken:  37 s,  5 m


In [ ]:
embedding_df = pd.DataFrame(embeds)
path = "drive/My Drive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/Questions/output.tsv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  embedding_df.to_csv(f, index=False, sep='\t')

In [ ]:
sentences_df = pd.DataFrame(question_sentences)
path = "drive/My Drive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/Questions/metadata.tsv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  sentences_df.to_csv(f, index=False, sep='\t')

In [ ]:
np.array(embedding_df).shape

(131123, 768)

In [ ]:
np.array(question_sentences).shape

(131123,)

In [ ]:
print("DONE")

##### Generating embeddings for answers

In [ ]:
t =time()
embeds = generate_embeddings("all-mpnet-base-v2", answer_sentences)
print("Time taken: " , int((time() - t)%60) ,"s, ", int((time() - t)/60), "m")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/file_download.py:563: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Time taken:  47 s,  5 m


In [ ]:
embedding_df = pd.DataFrame(embeds)
path = "drive/MyDrive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/Answers/output.tsv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  embedding_df.to_csv(f, index=False, sep='\t')

In [18]:
sentences_df = pd.DataFrame(answer_sentences)
path = "drive/MyDrive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/Answers/metadata.tsv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  sentences_df.to_csv(f, index=False, sep='\t')

In [ ]:
np.array(embedding_df).shape

In [ ]:
np.array(answer_sentences).shape

##### Finding product of the above generated embeddings